# Model testing

This notebook provides a set of tests to run against `treat_sim`.  These tests are either pass or fail and no interpretation is needed. A summary of test results is provided at the end of the notebook.

> This notebook is a work in progress.

We have broken the testing into the 

## 1. Model Code Imports

In [1]:
from treat_sim.model import (
    Scenario, 
    TreatmentCentreModel,
    single_run, 
    multiple_replications
)

## 2. Imports

In [2]:
import numpy as np
import pandas as pd
import statistics
import pytest
import ipytest
ipytest.autoconfig()

## 3. Tests

### 3.1 Model run test

Here we test that various modes of running the model work correctly.  These include

* single run mode
* repeatable results using random number sets.
* results collection period



In [3]:
def test_single_run_type():
    '''
    Test a single_run of the model.
    
    The single_run function should return a pandas.DataFrame
    containing 16 columns and a single row.
    
     0   00_arrivals                    
     1   01a_triage_wait                 
     2   01b_triage_util               
     3   02a_registration_wait         
     4   02b_registration_util        
     5   03a_examination_wait          
     6   03b_examination_util          
     7   04a_treatment_wait(non_trauma)  
     8   04b_treatment_util(non_trauma)  
     9   05_total_time(non-trauma)       
     10  06a_trauma_wait               
     11  06b_trauma_util                 
     12  07a_treatment_wait(trauma)      
     13  07b_treatment_util(trauma)      
     14  08_total_time(trauma)           
     15  09_throughput                   

    Expected result: 
    ---------------
        len(run_results) == 16 and isinstance(run_results, pd.Dataframe)

    Returns:
    -------
    bool: does the model pass the test.
    '''
    EXPECTED_LENGTH = 16

    # a default experiment
    default_experiment_params = Scenario(random_number_set=41)

    # run the model in single run model
    run_results = single_run(default_experiment_params)

    # test
    assert len(run_results.T) == EXPECTED_LENGTH and isinstance(run_results, pd.DataFrame)
    

In [4]:
@pytest.mark.parametrize('random_number_set', [
                          (0),
                          (1),
                          (2),
                          (101),
                          (42),
])
def test_random_number_set(random_number_set):
    '''
    Test the model produces repeatable results
    given the same set set of random seeds.
    
    Expected result: 
    ---------------
        difference between data frames is 0.0
    '''

    results = []

    for i in range(2):
        
        exp = Scenario()

        # run the model in single run model
        run_results = single_run(exp, random_no_set=random_number_set)
    
        results.append(run_results)
        
    # test
    assert (results[0] - results[1]).sum().sum() == 0.0

In [5]:
@pytest.mark.parametrize('rc_period', [
                          (10.0),
                          (1_000.0),
                          (25.0),
                          (500.0),
                          (143.0),
])
def test_run_length_control(rc_period):
    scenario = Scenario()
    
    # set random number set - this controls sampling for the run.
    scenario.set_random_no_set(42)

    # create an instance of the model
    model = TreatmentCentreModel(scenario)

    # run the model
    model.run(results_collection_period=rc_period)
        
    # run results
    assert model.env.now == rc_period

### 3.2. Extreme value tests

Here we manipulate the input parameters of the model to test that it behaves as expected. We run the following tests

* Zero arrivals of both types
* All arrivals are trauma
* All arrivals are non-trauma
* Infinite capacity for activities
* All non-trauma patients require treatment
* All non-trauma patients do not require treatment

> To investigate: blocked queues.  Simpy requires you to set resource count >=1; so cannot be simply blocked.  One way around this an arrival at time 0 and set activity time of the key activity to $M$ wa very large number. This means that the 1st patient arrives before the "real" arrival process begins, takes the 1 resource available and blocks the queue.  Do we actually need to do this?


In [38]:
### NOTE: we are ignoring mean of empty array warnings from numpy
### We will handle this in a future release of treat_sim.

@pytest.mark.filterwarnings("ignore")
@pytest.mark.parametrize('random_no_set', [
                          (42),
                          (1),
                          (754),
                          (9876534321),
                          (76546783986555),
])
def test_all_trauma(random_no_set):
    
    # create a new scenario and set prob of trauma to 100%
    scenario = Scenario(prob_trauma=1.0)

    # run the model in single run model
    run_results = single_run(scenario, random_no_set=random_no_set)
        
    # run results
    assert pd.isna(run_results['05_total_time(non-trauma)'].iloc[0]) and pd.isna(run_results['02a_registration_wait'].iloc[0]) and not pd.isna(run_results['08_total_time(trauma)'].iloc[0]) 

### 3.3 Deterministic activities

We have simplifed the model to a deterministic run by replacing all activity distributions with a fixed static value.

> To do: This will involve modifying the Scenario class distributions. Do we allow arrivals to remain stochastic (rate and patient type?). 

## 4. Run all automated tests

In [39]:
ipytest.run("-vv", "--no-header")

======================================= test session starts ========================================
collecting ... collected 16 items

t_553dd3e420a94c1083322397e4898f02.py::test_single_run_type PASSED                           [  6%]
t_553dd3e420a94c1083322397e4898f02.py::test_random_number_set[0] PASSED                      [ 12%]
t_553dd3e420a94c1083322397e4898f02.py::test_random_number_set[1] PASSED                      [ 18%]
t_553dd3e420a94c1083322397e4898f02.py::test_random_number_set[2] PASSED                      [ 25%]
t_553dd3e420a94c1083322397e4898f02.py::test_random_number_set[101] PASSED                    [ 31%]
t_553dd3e420a94c1083322397e4898f02.py::test_random_number_set[42] PASSED                     [ 37%]
t_553dd3e420a94c1083322397e4898f02.py::test_run_length_control[10.0] PASSED                  [ 43%]
t_553dd3e420a94c1083322397e4898f02.py::test_run_length_control[1000.0] PASSED                [ 50%]
t_553dd3e420a94c1083322397e4898f02.py::test_run_length_control[2

<ExitCode.OK: 0>

In [23]:
scenario = Scenario(prob_trauma=1.0)

# run the model in single run model
run_results = single_run(scenario, random_no_set=42)

# run results
run_results['05_total_time(non-trauma)']

/home/tom/miniforge3/envs/stars_docs/lib/python3.10/site-packages/treat_sim/model.py:916: RuntimeWarning: Mean of empty slice.
  if getattr(p, metric) > -np.inf]).mean()
/home/tom/miniforge3/envs/stars_docs/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


rep
1   NaN
Name: 05_total_time(non-trauma), dtype: float64

In [22]:
run_results

,00_arrivals,01a_triage_wait,01b_triage_util,02a_registration_wait,02b_registration_util,03a_examination_wait,03b_examination_util,04a_treatment_wait(non_trauma),04b_treatment_util(non_trauma),05_total_time(non-trauma),06a_trauma_wait,06b_trauma_util,07a_treatment_wait(trauma),07b_treatment_util(trauma),08_total_time(trauma),09_throughput
rep,,,,,,,,,,,,,,,,
1,209.0,16.622674,0.527512,NaN,0.0,NaN,0.0,NaN,0.0,NaN,474.329761,1.058193,31.50246,0.577901,518.095263,21.0
